In [12]:
using DataFrames, CSV
using JuMP, Gurobi
using LinearAlgebra, Random, Printf, StatsBase, CategoricalArrays
using Plots, StatsPlots
using Distributions
using Tables

In [13]:
const GRB_ENV = Gurobi.Env()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-22


Gurobi.Env(Ptr{Nothing} @0x000000000d162100, false, 0)

Indices:
$$\begin{aligned}
    i&=\text{index of tyre choice}\\
    j&=\text{index of the current number of consecutive laps}\\
    k&=\text{index of lap number}\\
\end{aligned}$$


In [14]:
driverNumber = 4;
df = DataFrame(CSV.File("DATA/DRIVER" * string(driverNumber) * "/info.csv",header=true));
df=df[:,2:ncol(df)]
df =Matrix(df)'

c = df
# df = DataFrame(CSV.File("laptimes.csv",header=false));
# c = Matrix(df[2:end,2:end]);

Delta_t_ps = 22000;
# c

22000

In [15]:
I = size(df)[1] 
J = 70
L = 70

70

Data:

$$\begin{aligned}
    c_{i,j}&=\text{real number that specifies the amount of time it takes to finish a lap with tyre choice $i$ in the $j$-th consecutive lap} \\
    \Delta t_{ps}&=\text{real number that specifies the amount of time required for a pit stop} \\
\end{aligned}$$


Decision variables:
$$\begin{aligned}
    t_{ijl} &= \text{binary variable that specifies whether tyre $i$ is chosen for the $j$-th consecutive lap in lap $l$} \\
    y_{l} &= \text{binary variable that specifies whether a pit stop is required in lap $l$} \\
\end{aligned}$$

In [16]:
model = Model(() -> Gurobi.Optimizer(GRB_ENV));
@variable(model, t[1:I,1:J,1:L], Bin);
@variable(model, y[1:L], Bin);

Objective Function:
$$\begin{equation}
    \text{minimize} \displaystyle \sum_{l = 1}^{L} \left( \Delta t_{ps} y_{l} + \sum_{i = 1}^{I} \sum_{j = 1}^{J} c_{ij} t_{ijl} \right) 
\end{equation}$$

In [17]:
@objective(model, Min, sum(Delta_t_ps*y[l] + sum(c[i,j]*t[i,j,l] for i=1:I, j=1:J) for l=1:L));

Constraints:

$$\begin{align}
y_{1} &= 0 && \\
\sum_{i = 1}^{I} t_{i11} &= 1\\
\sum_{i = 1}^{I} \sum_{j = 1}^{J} t_{ijl} &= 1 && \forall l \in 2, \dots, L \\
\sum_{l = 1}^{L} t_{2,1,l} &\leq 2 && \\
y_{l} &= \sum_{i = 1}^{I} t_{i1l} && \forall l\in 2, \dots, L \\
t_{ijl} &\leq t_{i,j-1,l-1} && \forall i \in 1, \dots, I, \forall j \in 2, \dots J, \forall l \in 2, \dots, L \\
t_{ijl} &= \{ 0, 1\} && \forall i \in 1, \dots, I, \forall j \in 1, \dots J, \forall l \in 1, \dots, L \\
y_{l} &= \{ 0, 1\} && \forall l \in 2, \dots, L
\end{align}$$

In [18]:
@constraint(model, constraint1[1], y[1] == 0);
@constraint(model, constraint2[1], sum(t[i,1,1] for i = 1:I) == 1);
@constraint(model, constraint3[l in 2:L], sum(t[i,j,l] for i = 1:I,j = 1:J) == 1);
@constraint(model, constraint4[i=1:I], sum(t[i,1,l] for l = 1:L) <= 1); # for all tyres you can you the at most once
@constraint(model, constraint5[l in 2:L], sum(t[i,1,l] for i = 1:I) == y[l]);
@constraint(model, constraint6[i in 1:I, j in 2:J, l in 2:L], t[i,j,l] <= t[i,j-1,l-1]);

┌ Warning: Axis contains one element: 1. If intended, you can safely ignore this warning. To explicitly pass the axis with one element, pass `[1]` instead of `1`.
└ @ JuMP.Containers /home/christopher/.julia/packages/JuMP/UqjgA/src/Containers/DenseAxisArray.jl:173
┌ Warning: Axis contains one element: 1. If intended, you can safely ignore this warning. To explicitly pass the axis with one element, pass `[1]` instead of `1`.
└ @ JuMP.Containers /home/christopher/.julia/packages/JuMP/UqjgA/src/Containers/DenseAxisArray.jl:173


In [19]:
optimize!(model);

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 62046 rows, 63770 columns and 188466 nonzeros
Model fingerprint: 0xf0013f1b
Variable types: 0 continuous, 63770 integer (63770 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+04, 1e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 887 rows and 913 columns
Presolve time: 0.47s
Presolved: 61159 rows, 62857 columns, 186680 nonzeros
Variable types: 0 continuous, 62857 integer (62857 binary)
Found heuristic solution: objective 7162925.0390
Found heuristic solution: objective 6388758.5963

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.01s

Solved with dual simplex

Root relaxation: objective 6.022511e+06, 6515 iterations, 1.07 seconds (1.01 work units)

    Nodes    |    Current Node    |   

## Retrieving the results

In [20]:
obj_opt = objective_value(model);
obj_opt

6.023123076345878e6

In [21]:
y_opt = value.(y);
t_opt = value.(t);
CSV.write("DATA/DRIVER" * string(driverNumber) * "/OptimalPitStopSchedule.csv",  Tables.table(y_opt), writeheader=false);


for i in 1:I 
    FileName = "DATA/DRIVER" * string(driverNumber) * "/Tyre-" * string(i) * ".csv";
    CSV.write(FileName,  Tables.table(t_opt[i,:,:]), writeheader=false);
end


In [22]:
t_opt = value.(t);

for l in 1:L #for every lap
    println("Lap: "*string(l))
    for i in 1:I
        for j in 1:J
            if t_opt[i, j, l] == 1
                println(i)
            end
        end
    end    
end
    

Lap: 1
5
Lap: 2
5
Lap: 3
5
Lap: 4
5
Lap: 5
5
Lap: 6
5
Lap: 7
5
Lap: 8
5
Lap: 9
5
Lap: 10
5
Lap: 11
5
Lap: 12
8
Lap: 13
8
Lap: 14
8
Lap: 15
8
Lap: 16
8
Lap: 17
8
Lap: 18
8
Lap: 19
8
Lap: 20
8
Lap: 21
8
Lap: 22
8
Lap: 23
8
Lap: 24
8
Lap: 25
8
Lap: 26
8
Lap: 27
11
Lap: 28
11
Lap: 29


11
Lap: 30
11
Lap: 31
11
Lap: 32
11
Lap: 33
11
Lap: 34
11
Lap: 35
11
Lap: 36
11
Lap: 37
11
Lap: 38
11
Lap: 39
11
Lap: 40
11
Lap: 41
11
Lap: 42
11
Lap: 43
11
Lap: 44
11
Lap: 45
11
Lap: 46
11
Lap: 47
11
Lap: 48
11
Lap: 49
11
Lap: 50
10
Lap: 51
10
Lap: 52
10
Lap: 53
10
Lap: 54
10
Lap: 55
10
Lap: 56
10
Lap: 57
10
Lap: 58
10
Lap: 59
10
Lap: 60
10
Lap: 61
10
Lap: 62
10
Lap: 63
10
Lap: 64
10
Lap: 65
10
Lap: 66
10
Lap: 67
10
Lap: 68
10
Lap: 69
10
Lap: 70
10
